# Import Python Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Read in Data
Some initial preprocessing was done:
* Replaced blanks with NAs (NaN in df below)
* Moved the reponse variable (revenue) to the first field

In [4]:
df = pd.read_csv('data.csv')
df.head()

,revenue,Week,weekno,highseason,holiday,displayimp,displaylag,brandpaid,brandclicks,nonbrandpaid,...,emailclick,blogvisits,blognewusers,blogrefer,blogrefernew,blogsignup,autoenroll,socialperson,promoteposts,learner
0,178480,2/2/14,1,1,0,987718,983958.0000,31589,4715,105901,...,2969,2802,1928,303,124,18,1,1,0,0
1,185390,2/9/14,2,1,0,660136,907388.9167,30428,4540,108652,...,3278,2940,2034,277,106,16,1,1,0,0
2,178130,2/16/14,3,1,0,604606,852862.4167,30715,4417,119398,...,2907,4359,3251,412,225,21,1,1,0,0
3,185020,2/23/14,4,1,0,560656,851885.3333,31632,4372,134819,...,3557,7408,6132,970,553,19,1,1,0,0
4,170950,3/2/14,5,1,0,326112,827164.4167,NaN,4431,115865,...,2455,2616,1752,310,111,11,1,1,0,0
